In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import json
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np
import pandas as pd

data preparation

In [5]:
df = pd.read_csv('news_data.csv')
print(len(df))
len(df)
corpus = [x for x in df.to_dict(orient='index').values()]
docs = [row['title'] for row in corpus if len(str(row.get('title'))) > 10] # avoid the empty or wrong docs

print(len(docs))
len(docs)
    
    

1969
1968


1968

TF IDF query processing

In [24]:

vectorizer = TfidfVectorizer()
tfidf_docs = vectorizer.fit_transform(docs)
print(tfidf_docs.shape, len(vectorizer.vocabulary_))
print(list(vectorizer.vocabulary_.keys())[:10])


(1968, 3166) 3166
['حمایت', 'واشنگتن', 'از', 'تلاش', 'های', 'دیپلمات', 'اروپایی', 'آغاز', 'نبرد', 'بر']


In [25]:
# query
def find_query(query : str = None):
  result = list()
  tfidf_query = vectorizer.transform([query])[0]

  # similarities
  cosines = []
  for doc  in tqdm(tfidf_docs):
    cosines.append(float(cosine_similarity(doc, tfidf_query)))
    
  # sorting
  k = 10
  sorted_ids = np.argsort(cosines)
  for i in range(k):
    cur_id = sorted_ids[-i-1]
    print((docs[cur_id], round(cosines[cur_id], 3) ))
    result.append((docs[cur_id], round(cosines[cur_id], 3) ))
  return result
  

In [27]:
queries = ['برنامه یارانه', 'سطح فقر', 'نتبجه مزاکرات برجام', 'ارز ۴۲۰۰', 'کنترل تورم در بازار']
result = []
for q in queries:
    result = find_query(q)
    df = pd.DataFrame(data = {'similarity': [r[1] for r in result] , 'doc_title': [r[0] for r in result ]})
    df.to_excel(f'samples/{q}.xlsx')

    


1968it [00:04, 464.53it/s]


('برنامه تکراری مجلس برای یارانه\u200cها', 0.641)
('برنامه دولت برای اصلاح نظام پرداخت یارانه', 0.57)
('عدالت در پرداخت یارانه\u200cها', 0.362)
('موضع رئیس\u200cجمهور درباره یارانه\u200cها', 0.354)
('روایت مجلس از برنامه حذف «4200»', 0.339)
('افزایش یارانه نقدی منتفی شد', 0.323)
('تکذیب صدور کارت یارانه نان', 0.31)
('برنامه مکتوب حذف ارز 4200', 0.304)
('اصلاح نظام برنامه\u200cوبودجه کشور', 0.296)
('تعطیلی برنامه تلویزیونی «نگاه یک»', 0.291)


1968it [00:03, 519.55it/s]


('دستور رئیسی برای رفع فقر مطلق', 0.358)
('برنامه دولت برای رفع فقر مطلق', 0.354)
('هجدهمین دور مذاکرات آستانه در سطح وزرای خارجه در ترکیه', 0.311)
('کرونا همچنان قربانی می\u200cگیرد', 0.0)
('راز چرخش برجامی روسیه', 0.0)
('سیاست متوازن در جنگ اوکراین', 0.0)
('محکومیت حمله موشکی رژیم صهیونیستی به حومه دمشق', 0.0)
('امضای سند همکاری\u200c\u200c\u200cهای مشترک امنیتی میان ایران و ازبکستان', 0.0)
('گفت\u200cوگوی تلفنی وزرای خارجه ایران و جمهوری آذربایجان', 0.0)
('«نور2» در مدار زمین', 0.0)


1968it [00:03, 502.03it/s]


('بسترسازی احیای برجام', 0.431)
('نشانه\u200cهای احیای برجام', 0.427)
('بازیگر جدید احیای برجام', 0.398)
('نظام بانکی و احیای برجام', 0.386)
('خوش\u200cبینی به احیای برجام', 0.375)
('ارزیابی روند مذاکرات احیای برجام', 0.374)
('کمپین قطر برای احیای برجام', 0.373)
('احیای برجام تا آخر هفته؟', 0.373)
('تصمیم جدید دولت آمریکا درباره برجام', 0.371)
('طرح اروپا برای نجات برجام', 0.37)


1968it [00:03, 610.40it/s]


('جلسات\u200c قالیباف\u200c با \u200cرئیسی \u200cدرباره حذف ارز ۴۲۰۰ تومانی', 0.56)
('تبعات حذف ارز 4200', 0.259)
('\u200cشرط مجلس برای حذف ارز 4200', 0.253)
('نگرانی مجلس از حذف ارز 4200', 0.252)
('تکذیب حذف ارز ترجیحی دارو', 0.248)
('اهداف حذف ارز 4200 در بودجه', 0.236)
('برنامه مکتوب حذف ارز 4200', 0.233)
('سردرگمی در اجرای حذف ارز 4200', 0.229)
('شرط موافقت با حذف ارز ترجیحی', 0.227)
('ارز دارو و گندم حذف نمی\u200cشود', 0.222)


1968it [00:03, 584.97it/s]


('کنترل مستمر بازار ضروری است', 0.494)
('دولت نسبت به تورم حساس است', 0.305)
('درخواست کنترل مهاجرت به پایتخت', 0.297)
('وعده قالیباف؛ تورم کاهش می\u200cیابد', 0.277)
('تذکر به دولت برای تعیین حقوق متناسب با تورم', 0.26)
('وضعیت خوب بازار سرمایه در چند هفته اخیر', 0.248)
('کنترل نامحسوس سفرهای خارجی مسوولان', 0.248)
('پیام رئیس\u200cجمهورایران برای امنیت بازار گاز', 0.237)
('دستگیری 5 اخلال\u200cگر شاخص بازار ارز', 0.218)
('بازار پرندگان وحشی فریدونکنار بسته شد', 0.217)
